<a href="https://colab.research.google.com/github/ashutoshghodke/ML_PROJECT/blob/main/pysparkimp25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=ffde2bea692f7a8ec699b81de4f1ccd3ecdcd364601b155137765abb405c19c1
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('practical').master('local[*]').getOrCreate()

In [5]:
print(spark)

In [6]:
rdd=spark.sparkContext.parallelize([1,2,3,4])

In [7]:
rdd.collect()

[1, 2, 3, 4]

In [8]:
rdd1=rdd.map(lambda x:x**2)

In [9]:
rdd1.collect()

[1, 4, 9, 16]

In [10]:
rdd2=["This is an sample application"]
v=spark.sparkContext.parallelize(rdd2)
rdd3=v.flatMap(lambda x:x.split(' '))

In [11]:
rdd3.collect()

['This', 'is', 'an', 'sample', 'application']

In [12]:
a = [1,2,3,4,5,6]
b = [7,8,9,4,5,6]

In [13]:
a1=spark.sparkContext.parallelize(a)
b1=spark.sparkContext.parallelize(b)

In [14]:
c=a1.union(b1)
c.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 4, 5, 6]

In [15]:
a1.intersection(b1).collect()

[4, 5, 6]

In [16]:
c.distinct().collect()

[4, 8, 1, 5, 9, 2, 6, 3, 7]

In [17]:
### groupByKey
x = spark.sparkContext.parallelize([("a",1),("s",5),("a",5),("s",4)])


In [20]:
x1=x.groupByKey().mapValues(len)


In [21]:
x1.collect()

[('s', 2), ('a', 2)]

In [23]:
## reduceByKey
x2 = spark.sparkContext.parallelize([("a",1),("s",5),("a",5),("s",4)])
x3=x2.reduceByKey(lambda x,y:x+y)
x3.collect()

[('s', 9), ('a', 6)]

In [28]:
## 11 sortByKey():
x4 = spark.sparkContext.parallelize([("a",1),("s",5),("a",5),("s",4)])
x5=x4.sortByKey()
x5.collect()

[('a', 1), ('a', 5), ('s', 5), ('s', 4)]

In [29]:
# 12 join():
#       it combines fields from two table using copmmon values

x = spark.sparkContext.parallelize([("a",1),("s",5),("a",5),("s",4)])
y = spark.sparkContext.parallelize([("a",1),("s",5),("a",5),("s",4)])
n=x.join(y)
n.collect()

[('s', (5, 5)),
 ('s', (5, 4)),
 ('s', (4, 5)),
 ('s', (4, 4)),
 ('a', (1, 1)),
 ('a', (1, 5)),
 ('a', (5, 1)),
 ('a', (5, 5))]

In [37]:
# 13 coalesce():
#          to avoid full shuffling of data we use coalesce() function.

x = spark.sparkContext.parallelize(["jan",'feb','mar','april','may','jun'],4)
a=x.coalesce(3)          #====> 3 represent no. of partittion
a.collect()
a.getNumPartitions()


3

In [ ]:
# The coalesce will decrease the number of partitions
# The repartition will increase the number of partitions


In [39]:
# """13 repartition()"""
b=a.repartition(8)
b.collect()
b.getNumPartitions()

8

In [40]:
a = ["001,Yash,Patil,29,M,011,20000",
"002,Ram,Wagh,30,M,012,30000",
"003,Sita,Patil,29,F,011,50000",
"004,Kiran,More,33,M,011,40000",
"005,Sachin,Tendulkar,51,M,012,100000"]

In [41]:
rdd = spark.sparkContext.parallelize(a)


In [42]:
# genderwise count
rdd.map(lambda x:x.split(",")).map(lambda x: (x[4],1)).reduceByKey(lambda x,y:x+y).collect()


[('M', 4), ('F', 1)]

In [43]:
# genderwise salary
rdd.map(lambda x:x.split(",")).map(lambda x: (x[4],int(x[6]))).reduceByKey(lambda x,y:x+y).collect()


[('M', 190000), ('F', 50000)]

In [46]:
# second highest salary

a =rdd.map(lambda x:x.split(",")).map(lambda x:int(x[6]))
c = a.max()
print(c)

print(a.filter(lambda x:x != c).max())



100000
50000


In [ ]:
# # broadcast variables
# broadcast varibles are read only shared variables that are cached and available on all nodes in cluster
# in order to access by the task.

# accumulator
# Accumulators are write only shared variables and used to perform add and counter operations


In [48]:
# # broadcast variables

data = [("James","Smith","USA","CA"),
    ("Michael","Rose","USA","NY"),
    ("Robert","Williams","USA","CA"),
    ("Maria","Jones","USA","FL")
  ]

states = {"NY":"New York", "CA":"California", "FL":"Florida"}

b_s = spark.sparkContext.broadcast(states)

def states_con(code):
    return b_s.value[code]

rdd = spark.sparkContext.parallelize(data)

rdd.map(lambda x: (x[0],x[1],x[2],states_con(x[3]))).collect()



[('James', 'Smith', 'USA', 'California'),
 ('Michael', 'Rose', 'USA', 'New York'),
 ('Robert', 'Williams', 'USA', 'California'),
 ('Maria', 'Jones', 'USA', 'Florida')]

In [50]:
# accumulator
accu = spark.sparkContext.accumulator(0)

a = [1,2,3,4]
rdd = spark.sparkContext.parallelize(a)

rdd.foreach(lambda x: accu.add(x))

print(accu.value)


10


In [51]:
# pivot()
data = [("Banana",1000,"USA"), ("Carrots",1500,"USA"), ("Beans",1600,"USA"), \
      ("Orange",2000,"USA"),("Orange",2000,"USA"),("Banana",400,"China"), \
      ("Carrots",1200,"China"),("Beans",1500,"China"),("Orange",4000,"China"), \
      ("Banana",2000,"Canada"),("Carrots",2000,"Canada"),("Beans",2000,"Mexico")]

columns= ["Product","Amount","Country"]
df = spark.createDataFrame(data = data, schema = columns)
df.show()

+-------+------+-------+
|Product|Amount|Country|
+-------+------+-------+
| Banana|  1000|    USA|
|Carrots|  1500|    USA|
|  Beans|  1600|    USA|
| Orange|  2000|    USA|
| Orange|  2000|    USA|
| Banana|   400|  China|
|Carrots|  1200|  China|
|  Beans|  1500|  China|
| Orange|  4000|  China|
| Banana|  2000| Canada|
|Carrots|  2000| Canada|
|  Beans|  2000| Mexico|
+-------+------+-------+



In [52]:
df.groupBy('product').pivot('country').sum('amount').show()


+-------+------+-----+------+----+
|product|Canada|China|Mexico| USA|
+-------+------+-----+------+----+
| Orange|  null| 4000|  null|4000|
|  Beans|  null| 1500|  2000|1600|
| Banana|  2000|  400|  null|1000|
|Carrots|  2000| 1200|  null|1500|
+-------+------+-----+------+----+



In [53]:
# 1 row_number() --used to give sequential number

from pyspark.sql.window import Window

a = Window.partitionBy('dept').orderBy(col('salary').desc())

sh = df.withColumn('number',row_number().over(a))
sh.show()

NameError: ignored

In [ ]:
#rank()  --used to give ranking .
# --if same values are there it gives them same rank and
#         for the next value rank will be summation of previous rank +
#         no. of duplicates of previous rank

r = df.withColumn('rank',rank().over(Window.orderBy(col('salary').desc()))).show()


In [ ]:
# dense_rank() ---used to give ranking

#   if there are same values it gives same rank to them & for next value it gives next rank


a = Window.partitionBy('dept').orderBy(col('salary').desc())

df.withColumn('ranking',dense_rank().over(a)).where(col('ranking')==2).show()



In [ ]:
# joins

# inner join - matching records
# df.join(df1, df.name == df1.name,'inner').show()


# left/ left_outer
# df.join(df1,df.name == df1.name, 'left').show()

# right / right_outer
df.join(df1, df.name == df1.name, 'right').show()


# leftsemi
# df.join(df1,df.name == df1.name, 'leftsemi').show()    # ignores right dataset

# leftanti
# df.join(df1, df.name == df1.name , 'leftanti').show()   # returns non matching recoerds from left dataset

# full / full_outer
# df.join(df1, df.name == df1.name , 'full').show()


In [ ]:
#cast

df1 = df.select(df.bonus.cast('string'))

df1.printSchema()

In [ ]:
# withcolumn

# cast()    -change datatype of column
df1 = df.withColumn('salary',col('salary').cast('Integer'))

# add new column using existing column
df.withColumn('salarys', col('salary')*10).show()

# lit()  - add new column with default values
df.withColumn('none',lit('null')).show()

In [ ]:

#count()
df.groupBy('dept').count().show()

# sum()
# df.groupBy('state').sum('bonus').show()

# max()
# df.groupBy('dept').max('salary').show()

# min()
# df.groupBy('dept').min('salary').show()

# avg()
# df.groupBy('dept').avg('salary').show()


#mean()
# df.groupBy('dept').mean('salary').show()


# .agg

# df.groupBy('dept').agg(sum('salary').alias('total'),
#                        count('salary').alias('count'),
#                       max('salary').alias('max'),
#                       min('salary').alias('min'),
#                       avg('salary').alias('avg')).show()

In [ ]:
# salary greater than avg sal
"""1"""
df1 = df.select(avg('salary').cast('Integer')).collect()[0][0]
print(df1)

df.filter(df.salary > df1).show()